In [ ]:
## SEQUENCING SPARE PARTS

In [ ]:
SENSOR_RECEPTIVE_FIELD_SIZE = 8
SENSOR_RECEPTIVE_FIELD_SQUARE = SENSOR_RECEPTIVE_FIELD_SIZE ** 2

SENSORS_COUNT = config.sensors_count
SENSORS_COUNT_ROOT = int(np.sqrt(SENSORS_COUNT))
assert SENSORS_COUNT_ROOT ** 2 == SENSORS_COUNT
assert SENSORS_COUNT_ROOT > 1

In [ ]:
SENSOR_STATES_INFO = defaultdict(list)
SENSOR_STATES_IMG = []
sz = SENSOR_RECEPTIVE_FIELD_SIZE
captions = []

assert config.encoding_type == 'sequence'

In [ ]:
angle_step = 10 # degrees
angles = list(range(0, 360, angle_step)) + [45 * 1, 45 * 3, 45 * 5, 45 * 7]
angles = sorted(angles)
# sweeps = [50, 90, 130, 180, 230, 270, 310] # for receptive field size = 8 more finer sweeps make little sense 
sweeps = [90, 130, 180, 230, 270] # for receptive field size = 8 more finer sweeps make little sense 

for angle in angles:
    # for sweep in range(angle_step, 180 + angle_step, angle_step):
    for sweep in sweeps:
        canvas = Image.new('L', (sz, sz))
        draw = ImageDraw.Draw(canvas)
        draw.rectangle([0, 0, sz, sz], fill=127, outline=127)
        piesclice_angle1 = angle + sweep // 2
        piesclice_angle2 = angle - sweep // 2
        draw.pieslice([-sz, -sz, sz * 2 - 1, sz * 2 - 1], start=piesclice_angle1, end=piesclice_angle2, fill=255, outline=255, width=0)
        captions.append(f'ang={angle}, swp={sweep} ')
        SENSOR_STATES_IMG.append(canvas)
        SENSOR_STATES_INFO['normal'].append(angle)
        SENSOR_STATES_INFO['sweep'].append(sweep)
        SENSOR_STATES_INFO['normal_vec'].append(np.exp((1j) * np.radians(angle)))
        SENSOR_STATES_INFO['sweep1_vec'].append(np.exp((1j) * np.radians(piesclice_angle1)))
        SENSOR_STATES_INFO['sweep2_vec'].append(np.exp((1j) * np.radians(piesclice_angle2)))

SENSOR_STATES_INFO = pd.DataFrame(SENSOR_STATES_INFO)
assert len(SENSOR_STATES_INFO) == len(SENSOR_STATES_IMG)
# n_to_show = len(sweeps) * 4
# images_to_show = list(map(lambda x: lay_grid(x.resize((80, 80)), 8), SENSOR_STATES_IMG[:n_to_show]))
# display_images_grid(images_to_show, captions=captions[:n_to_show], col_count=len(sweeps) )

In [ ]:
# Turn grayscale images of sensor states to numpy arrays with weights which balance positive and negative areas
SENSOR_STATES = np.array(list(map(np.array, SENSOR_STATES_IMG))).astype(float)
SENSOR_STATES = SENSOR_STATES.reshape(-1, SENSOR_RECEPTIVE_FIELD_SIZE ** 2)
SENSOR_STATES[SENSOR_STATES == 255] = 1
SENSOR_STATES[SENSOR_STATES == 127] = -1
assert set(np.unique(SENSOR_STATES)) == set([-1, +1])

SENSOR_STATES_COUNT = SENSOR_STATES.shape[0]
# SENSOR_STATES.shape, np.unique_counts(SENSOR_STATES)

In [ ]:
SENSOR_STATE_AREAS_POS = (SENSOR_STATES == 1).sum(axis=1) 
SENSOR_STATE_AREAS_NEG = (SENSOR_STATES == -1).sum(axis=1)
assert np.all(SENSOR_STATE_AREAS_POS > 0)
assert np.all(SENSOR_STATE_AREAS_NEG > 0)
# SENSOR_STATE_AREAS_POS.shape, SENSOR_STATE_AREAS_POS.mean(), SENSOR_STATE_AREAS_POS.std(), \
# SENSOR_STATE_AREAS_NEG.shape, SENSOR_STATE_AREAS_NEG.mean(), SENSOR_STATE_AREAS_NEG.std()

In [ ]:
SENSOR_STATES_POS = xp_array_to_gpu_copy(SENSOR_STATES)
SENSOR_STATES_NEG = xp_array_to_gpu_copy(SENSOR_STATES)
SENSOR_STATES_POS[SENSOR_STATES_POS < 0] = 0
SENSOR_STATES_NEG[SENSOR_STATES_NEG > 0] = 0

In [ ]:
class SensorLayoutMaker(object):
    def __call__(self, sensor_index):
        return None

class SensorLayoutMaker_Grid(SensorLayoutMaker):
    def __init__(self):
        self.s = SENSORS_COUNT_ROOT
        self.field_size = SENSOR_RECEPTIVE_FIELD_SIZE
        space_for_sensors = self.s * self.field_size 
        self.dist_between_sensors = (config.sample_size - space_for_sensors) / (self.s - 1)
        
    def __call__(self, sensor_index):
        x = int((sensor_index % self.s) * (self.field_size + self.dist_between_sensors))
        y = int((sensor_index // self.s) * (self.field_size + self.dist_between_sensors))
        # adjust to fit into boundaries
        x -= max(0, (x + self.field_size) - config.sample_size) 
        y -= max(0, (y + self.field_size) - config.sample_size)
        return x, y, self.field_size

In [ ]:
SENSOR_LAYOUT_MAKER = None
    
match config.retina_layout:
    case 'grid': 
        SENSOR_LAYOUT_MAKER = SensorLayoutMaker_Grid()
    case _:
        assert False, config.retina_layout

In [ ]:
assert not SENSOR_LAYOUT_MAKER is None
SENSOR_INSTANCES_INFO = pd.DataFrame(columns=['x_offset', 'y_offset', 'size', 'bounds_rect', 'x_center', 'y_center', 'radius'])

for i in range(SENSORS_COUNT):
    x_offset, y_offset, field_size = SENSOR_LAYOUT_MAKER(i)
    assert x_offset >= 0 and x_offset <= config.sample_size - field_size, (i, x_offset, field_size)
    assert y_offset >= 0 and y_offset <= config.sample_size - field_size, (i, y_offset, field_size)
    SENSOR_INSTANCES_INFO.loc[len(SENSOR_INSTANCES_INFO)] = [
        x_offset,
        y_offset,
        field_size,
        [x_offset, y_offset, x_offset + field_size - 1, y_offset + field_size - 1],
        x_offset + field_size // 2,
        y_offset + field_size // 2,
        field_size // 2
    ]

# SENSOR_INSTANCES_INFO

In [ ]:
SENSOR_PATCH_COORDS = []
    
for row in SENSOR_INSTANCES_INFO.itertuples():
    x_offset = row.x_offset
    y_offset = row.y_offset
    size = row.size

    for i in range(size):
        y = (y_offset + i) * config.sample_size
        stride_coords = np.arange(y + x_offset, y + x_offset + size)
        SENSOR_PATCH_COORDS.append(stride_coords)

SENSOR_PATCH_COORDS = np.array(SENSOR_PATCH_COORDS)

In [ ]:
def get_image_patches_for_sensor_instances(images_matrix):
    assert images_matrix.ndim == 2, images_matrix.ndim
    assert images_matrix.shape[1] == config.sample_size ** 2, images_matrix.shape[1]
    images_count = images_matrix.shape[0]
    # Result is 3d-tensor: 1 dim - image, 2 dim - patch no, 3 dim - patch itself (pixels)
    return np.take(images_matrix, SENSOR_PATCH_COORDS, axis=1).reshape(images_count, -1, SENSOR_RECEPTIVE_FIELD_SQUARE)

In [ ]:
# Returns matrix of sense_vectors (one row = sense_vector for each image in images_matrix)
# Each elem in sense_vector describes sensor state of each sensor instance (-1 if sensor instance is not activated)
def sense_images(images_matrix):
    MINIMAL_ILLUMINATION_ABS_LEVEL = 255 * 0.1
    MINIMAL_ILLUMINATION_DIFF_DB = 2 # dB
    MINIMAL_ILLUMINATION_DIFF_RATIO = pow(10, MINIMAL_ILLUMINATION_DIFF_DB/20)
    MINIMAL_PLUS_MINUS_DISTR_REL_DIFF = 0.9
    
    assert images_matrix.ndim == 2
    images_count = images_matrix.shape[0]
    images_patches = get_image_patches_for_sensor_instances(images_matrix) # array of matrices: image no -> matrix with patches as row vectors
    
    mean_luminiscene_in_patches = images_patches.mean(axis=2) # matrix where each row = avg of luminisnece in each patch of an image
    mean_images_patches = images_patches.reshape(-1, SENSOR_RECEPTIVE_FIELD_SQUARE).T - mean_luminiscene_in_patches.ravel()
    mean_images_patches = mean_images_patches.T.reshape(images_count, -1, SENSOR_RECEPTIVE_FIELD_SQUARE).transpose(0, 2, 1)
    # mean_images_patches is an array of matrices: image no -> matrix with mean-centered patches as column vectors
    
    images_patches = images_patches.transpose(0, 2, 1) # array of matrices: image no -> matrix with patches as column vectors
    assert images_patches.shape == mean_images_patches.shape
    
    abs_illum_level_pos = (SENSOR_STATES_POS @ images_patches).transpose(0, 2, 1)
    abs_illum_level_neg = (SENSOR_STATES_NEG @ images_patches).transpose(0, 2, 1)
    rel_illum_level_pos = (SENSOR_STATES_POS @ xp.where(mean_images_patches > 0, +1, 0)).transpose(0, 2, 1) # for +/- tally
    rel_illum_level_neg = (SENSOR_STATES_NEG @ xp.where(mean_images_patches < 0, -1, 0)).transpose(0, 2, 1)  # for +/- tally
    
    assert abs_illum_level_pos.shape == (images_count, SENSORS_COUNT, SENSOR_STATES_COUNT), abs_illum_level_pos.shape
    assert abs_illum_level_neg.shape == abs_illum_level_pos.shape
    assert rel_illum_level_pos.shape == abs_illum_level_pos.shape
    assert rel_illum_level_neg.shape == abs_illum_level_pos.shape
    
    sense_vectors = []
    # For each patch in image we detect likelihood of activation of sensor in one of its (sensor's) state.
    # If we have some state (say some angle and some sweep), what is condition that given patch corresponds to this state?
    # Of course patch must be somewhat illuminated (regulated by SENSORS_POS) - this is to ensure that we don't get fooled by situation
    # when e.g. negative part is not illuminated (dot prod is 0) while positive part is illuminated very, very week (say dot prod is 5). Looks
    # like this is the reason why we don't hear very low frequencies or don't percieve low light - becase ratio get oversaturaed very quickly.
    # But the key point is that sum of pixels luminiscence can meet averaged criterion only when bright pixels are concentared in area covered by SENSORS_POS
    # and dark pixels are concentrated in area covered by SENSORS_NEG. Otherwise there will disbalance.
    # To detect this condition we can look at distribution of mean-centered patches: +pixels must reside in positive part, -pixels must reside in negative part.
    # To restassure that +/- distribution of pixels actually corresponds to detectable diff in illumination we also need to compare
    # absolute illumination on positive and negative part
    for ailp_i, ailn_i, rilp_i, riln_i in zip(abs_illum_level_pos, abs_illum_level_neg, rel_illum_level_pos, rel_illum_level_neg): # Per image cycle
        # ailp_i and ailn_i both are matrices N(patches count) x M(count of sensor states)
        shape_save = ailp_i.shape
    
        # MAC {0, 1}
        # Illumination level on positive area must exceed lower threshold (sensor must be somewhat illuminated)
        # abs_illumination_scores = matrix of N(patches count), columns - 0/1 if patch is minimally illuminated according to given sensor's state
        abs_illumination_scores = ailp_i > MINIMAL_ILLUMINATION_ABS_LEVEL * SENSOR_STATE_AREAS_POS
    
        # MAC {0, 1}
        # Illumination level on positive part must distinguishably exceed illumination level of negative part
        ailn_i += 1e-6 # to get rid of possible division by zero errors
        ailp_i = ailp_i / SENSOR_STATE_AREAS_POS
        ailn_i = ailn_i / SENSOR_STATE_AREAS_NEG
        diff_abs_illumination_scores = xp.abs(ailp_i.ravel() / ailn_i.ravel())
        diff_abs_illumination_scores = diff_abs_illumination_scores >= MINIMAL_ILLUMINATION_DIFF_RATIO
        # diff_abs_illumination_scores = matrix of N(patches count), columns - 0/1 if patch is illuminated more on positive part than on negative
        diff_abs_illumination_scores = diff_abs_illumination_scores.reshape(shape_save)
    
        # Score {0, [MINIMAL_PLUS_MINUS_DISTR_REL_DIFF...1]}
        # Tally concentration of +/- pixels on positive/negative side and compare against theirs areas. 
        # For sensors with proper distribution ratio will be around 1 for each of the scores.
        # *concentration_scores = matrix of N(patches count), columns - [0...1] how well pixels in patch are distributed according to given sensor's state
        pluses_concentration_scores = rilp_i / SENSOR_STATE_AREAS_POS
        # assert xp.all((pluses_concentration_scores >= 0) & (pluses_concentration_scores <= 1)) # comment for production (to speed up)
        mask = (pluses_concentration_scores < MINIMAL_PLUS_MINUS_DISTR_REL_DIFF)
        pluses_concentration_scores = xp.where(mask, 0, pluses_concentration_scores)
    
        minuses_concentration_scores = riln_i / SENSOR_STATE_AREAS_NEG
        # assert xp.all((minuses_concentration_scores >= 0) & (minuses_concentration_scores <= 1)) # comment for production (to speed up)
        mask = (minuses_concentration_scores < MINIMAL_PLUS_MINUS_DISTR_REL_DIFF)
        minuses_concentration_scores = xp.where(mask, 0, minuses_concentration_scores)
        
        # sense_matrix = matrix of N(patches count), columns - 0/1 if all three conditions meet for given sensor state
        sense_matrix = (abs_illumination_scores.astype(float) + diff_abs_illumination_scores.astype(float) + pluses_concentration_scores + minuses_concentration_scores)
        min_score = 1 + 1 + (MINIMAL_PLUS_MINUS_DISTR_REL_DIFF * 2)
        sense_vector = xp.where(xp.any(sense_matrix > min_score, axis=1), xp.argmax(sense_matrix, axis=1), -1)
        sense_vectors.append(sense_vector)
    
    return xp.vstack(sense_vectors)

In [ ]:
SensorInstance = namedtuple('SensorInstance', ['Index', 'sensor_instance_ind', 'sensor_state_ind', 'normal', 'sweep', 'radius', 
                                               'x_center', 'y_center', 'x_center_orig', 'y_center_orig', 'altitude', 'x'])

In [ ]:
# Returns (altitude_map, df_sensor_instances) for sense_vector
def parse_sense_vector(sense_vector):
    ALTITUDE_EXTEND_BY = 5 # number of new intermediate cells for each side of 2x2 rect, e.g. 3x3 becomes 13x13 if altitude_extend_by is 5
    MAX_LANDING_STEPS = 1000
    MAX_ALTITUDE_THRESHOLD = 2
    
    assert sense_vector.shape[0] == len(SENSOR_INSTANCES_INFO)
    sensor_instance_inds = np.argwhere(sense_vector != -1).ravel()
    sensor_state_inds = np.take(sense_vector, sensor_instance_inds)
    assert sensor_instance_inds.shape[0] == sensor_state_inds.shape[0]
    df_sensor_instances_raw = pd.DataFrame({'sensor_instance_ind': sensor_instance_inds, 'sensor_state_ind': sensor_state_inds})
    df_sensor_instances_raw = df_sensor_instances_raw.merge(SENSOR_STATES_INFO, how='left', left_on='sensor_state_ind', right_index=True)
    df_sensor_instances_raw = df_sensor_instances_raw.merge(SENSOR_INSTANCES_INFO, how='left', left_on='sensor_instance_ind', right_index=True)

    # Lo-resolution altitude map
    altitude_map_lores = np.zeros((config.sample_size, config.sample_size))

    for si in df_sensor_instances_raw.itertuples():
        sensor_state = SENSOR_STATES[si.sensor_state_ind]
        size = si.size
        altitude_map_lores[si.y_offset:si.y_offset + size, si.x_offset:si.x_offset + size] += sensor_state.reshape(size, size)
    
    altitude_map_lores_mask = np.full_like(altitude_map_lores, np.min(altitude_map_lores), dtype='i')
    
    for si in df_sensor_instances_raw.itertuples():
        altitude_map_lores_mask[si.y_offset:si.y_offset + size, si.x_offset:si.x_offset + size] = 0
    
    altitude_map_lores += altitude_map_lores_mask
    altitude_map_lores = altitude_map_lores.astype('i')

    # Enhance lo-resolution altitude map by interpolating. This is to develop low altitude areas which are 
    # hidden beyond patches like [[-4, -5], [3, 4]]
    altitude_lores_to_hires_translate_factor = (ALTITUDE_EXTEND_BY + 1)
    altitude_map_hires_size = (config.sample_size - 1) * altitude_lores_to_hires_translate_factor + 1
    altitude_map_hires = np.zeros((altitude_map_hires_size, altitude_map_hires_size), dtype='f')
    patch_hires_shape = (2 + ALTITUDE_EXTEND_BY, 2 + ALTITUDE_EXTEND_BY)
    ii, jj = np.mgrid[0:patch_hires_shape[0], 0:patch_hires_shape[1]]
    patch_hires_indices = np.array(list(zip(ii.ravel(), jj.ravel())))
    
    for i in range(altitude_map_lores.shape[0] - 1):
        for j in range(altitude_map_lores.shape[1] - 1):
            patch = altitude_map_lores[i:i+2, j:j+2].astype(float)
    
            if patch[0,0] == patch[0,1] and patch[0,1] == patch[1,0] and patch[1,0] == patch[1,1]:
                patch_hires = patch[0,0]
            else:
                interp = RegularGridInterpolator(([0, patch_hires_shape[0] - 1], [0, patch_hires_shape[1] - 1]), patch)
                patch_hires = interp(patch_hires_indices).reshape(patch_hires_shape)
                
            i_hires = i * altitude_lores_to_hires_translate_factor
            j_hires = j * altitude_lores_to_hires_translate_factor
            altitude_map_hires[i_hires:i_hires+patch_hires_shape[0], j_hires:j_hires+patch_hires_shape[1]] = patch_hires

    # Reposition sensor instances so they occur in a valley with low absolute altitude. Some of the sensor instances will get discarded
    landed_centers = defaultdict(list)

    for si in df_sensor_instances_raw.itertuples():
        x_center_land = si.x_center * altitude_lores_to_hires_translate_factor
        y_center_land = si.y_center * altitude_lores_to_hires_translate_factor
        altitude = altitude_map_hires[y_center_land, x_center_land]
    
        if np.abs(altitude) > 1:
            travel_point = complex(x_center_land, y_center_land)
            direction = -1 if altitude < 0 else +1
            
            for i in range(MAX_LANDING_STEPS):
                travel_point += direction * si.normal_vec
                new_x_center_land = np.round(travel_point.real, decimals=0).astype(int)
                new_y_center_land = np.round(travel_point.imag, decimals=0).astype(int)
                new_altitude = np.abs(altitude_map_hires[new_y_center_land, new_x_center_land])
    
                if new_altitude <= altitude:
                    altitude = new_altitude
                    x_center_land = new_x_center_land
                    y_center_land = new_y_center_land
                else:
                    break
            else:
                assert False
    
        landed_centers['x_center_new'].append(x_center_land)
        landed_centers['y_center_new'].append(y_center_land)
        landed_centers['x_center_orig'].append(int(x_center_land / altitude_lores_to_hires_translate_factor))
        landed_centers['y_center_orig'].append(int(y_center_land / altitude_lores_to_hires_translate_factor))
        landed_centers['altitude_new'].append(altitude)
    
    df_sensor_instances = df_sensor_instances_raw.copy()
    
    for col_name, values in landed_centers.items():
        df_sensor_instances[col_name] = values
    
    df_sensor_instances.drop(['x_center', 'y_center', 'x_offset', 'y_offset', 'bounds_rect', 'normal_vec', 'sweep1_vec', 'sweep2_vec', 'size'], axis=1, inplace=True)
    df_sensor_instances.rename({'x_center_new': 'x_center', 'y_center_new': 'y_center', 'altitude_new': 'altitude'}, axis=1, inplace=True, errors='raise')
    df_sensor_instances = df_sensor_instances.loc[np.abs(df_sensor_instances.altitude) < MAX_ALTITUDE_THRESHOLD]
    df_sensor_instances['x'] = df_sensor_instances.x_center.astype(str) + ',' + df_sensor_instances.y_center.astype(str)
    df_sensor_instances.drop_duplicates(subset=['x'], inplace=True)
    return (altitude_map_hires, df_sensor_instances)

In [ ]:
def discover_raw_sequences(sensor_instance_ind_dict, altitude_map):
    sensor_instance_map = dict(map(lambda si: ((si.y_center, si.x_center), si), sensor_instance_ind_dict.values()))
    unsorted_sensor_instance_inds = set(map(lambda si: si.Index, sensor_instance_ind_dict.values()))
    raw_sequences = {} # start_sensor_inst_index -> sensor_inst_chains
    
    while unsorted_sensor_instance_inds:
        # choose random sensor instance to start from
        start_sensor_inst_index = unsorted_sensor_instance_inds.pop()
        start_sensor_inst = sensor_instance_ind_dict[start_sensor_inst_index]
        
        wave_front_que = deque([(int(start_sensor_inst.y_center), int(start_sensor_inst.x_center))])
        wave_front_map = np.full_like(altitude_map, -1, dtype='i')
        wave_front_map[wave_front_que[0]] = start_sensor_inst.Index
        wave_front_touched = set([wave_front_que[0]])
        
        sensor_inst_chains = {} # key - last element in chain of sensor instances, value - chain itself
        
        # LOG(f'Starting countour from {start_sensor_inst_index} (of {len(unsorted_sensor_instance_inds)})')
        
        while wave_front_que:
            # Visit point on top of the wave_front
            wave_front_point = wave_front_que.popleft()
            wave_front_sensor_inst_ind = wave_front_map[wave_front_point]
            wave_front_map[wave_front_point] = -1
            assert wave_front_sensor_inst_ind > -1
            # LOG(f'Visiting {wave_front_point} (si={wave_front_sensor_inst_ind}) (of {len(wave_front_que)})')
        
            si = sensor_instance_map.get(wave_front_point, None)
        
            if not si is None and si.Index != wave_front_sensor_inst_ind:
                # New sensor instance discovered! Link with the previous one, ...
                sensor_inst_chain = sensor_inst_chains.pop(wave_front_sensor_inst_ind, [wave_front_sensor_inst_ind])
                sensor_inst_chain.append(si.Index)
                assert not si.Index in sensor_inst_chains, f'{si.Index} in sensor_inst_chains'
                sensor_inst_chains[si.Index] = sensor_inst_chain
                
                # LOG(f'New si encountered: {si.Index}, linked after {wave_front_sensor_inst_ind}')
                wave_front_sensor_inst_ind = si.Index
                # ... and adjust adjacent wave front points to account for new sensor instance
                # (adjacency criteria enables coexisting of separate (disjoint) wave fronts on a map)
                adjust_stack = [wave_front_point]
                adjust_touched = set([wave_front_point])
        
                while adjust_stack:
                    adjust_point = adjust_stack.pop()
        
                    for rel_ij in [[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1]]:
                        ij = int(adjust_point[0] + rel_ij[0]), int(adjust_point[1] + rel_ij[1])
                
                        if ij[0] < 0 or ij[1] < 0 or ij[0] >= wave_front_map.shape[0] or ij[1] >= wave_front_map.shape[1]:
                            continue
                        elif wave_front_map[ij] == -1:
                            continue
                        elif ij in adjust_touched:
                            continue
        
                        # LOG(f'Adjusting {ij} (si={wave_front_map[ij]}) to si={si.Index}')
                        wave_front_map[ij] = si.Index
                        adjust_stack.append(ij)
                        adjust_touched.add(ij)
        
            # Plan further spread of the wave front
            for rel_ij in [[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1]]:
                ij = int(wave_front_point[0] + rel_ij[0]), int(wave_front_point[1] + rel_ij[1])
                # LOG(f'... spread, checking coord {rel_ij} {ij}')
        
                if ij[0] < 0 or ij[1] < 0 or ij[0] >= altitude_map.shape[0] or ij[1] >= altitude_map.shape[1]:
                    # LOG(f'... spread, {ij} out of boundaries')
                    continue
                elif np.abs(altitude_map[ij]) >= 2: # coord_ij falls out of valley
                    # LOG(f'... spread, {ij} has bad altitude {altitude_map[ij]}')
                    continue
                elif ij in wave_front_touched:
                    # LOG(f'... spread, {ij} already touched ({len(wave_front_touched)})')
                    continue
        
                wave_front_touched.add(ij)
                wave_front_que.append(ij)
                wave_front_map[ij] = wave_front_sensor_inst_ind
                # LOG(f'... new wave front point {wave_front_que[-1]} (si={wave_front_sensor_inst_ind})')
    
        raw_sequences[start_sensor_inst_index] = sensor_inst_chains
        # Collect all sensor instances encountered during this run and account them as sorted
        sorted_sensor_instance_inds = set(itertools.chain.from_iterable(sensor_inst_chains.values()))
        unsorted_sensor_instance_inds -= sorted_sensor_instance_inds

    return raw_sequences

In [ ]:
def weave_raw_sequences(raw_sequences):
    sequences = []
        
    for sensor_inst_chains in raw_sequences.values():
        # Our goal is to find two longest subsequence which start from the same si and clue them together. 
        # Usually this would be two subsequences starting from start_sensor_inst_index.
        # This way we would get longest serial sequence of traverse
        subsequences = []
        
        for chain_index, chain in enumerate(sensor_inst_chains.values()):
            chains_wo_chain_index = map(lambda v: v[1], filter(lambda v: v[0] != chain_index, enumerate(sensor_inst_chains.values())))
            chains_dict = defaultdict(list)
            for v in chains_wo_chain_index: chains_dict[v[0]].append(v)
            
            subsequences.append([])
            branch_stack = [(chain, 0, len(subsequences) - 1)]
            # LOG(f'Starting new subsequence #{len(subsequences) - 1}, {chain[0]}')
        
            while branch_stack:
                chain, chain_elem_index, subseq_index = branch_stack.pop()
                subseq = subsequences[subseq_index]
                # LOG(f'Weaving subsequence #{subseq_index}, {chain[chain_elem_index]}')
                
                for i in range(chain_elem_index, len(chain)):
                    v = chain[i]
                    subseq.append(v)
                    # LOG(f'... adding {chain[i]}, subseq len={len(subseq)}, subseq[:3]={subseq[:3]}, subseq[-3:]={subseq[-3:]}')
        
                    if v in chains_dict:
                        # LOG(f'... branch point detected')
                        branch_stack.append((chain, i + 1, subseq_index)) # continue sequence by following current chain
    
                        # Start new subsequences from new chain
                        for branch_chain in chains_dict[v]:
                            new_subseq = subseq.copy()
                            subsequences.append(new_subseq)
                            branch_stack.append((branch_chain, 1, len(subsequences) - 1)) # create new sequence and follow new chain
                            # LOG(f'... branching new subsequence #{len(subsequences) - 1}, {branch_chain[1]}')
                            
                        break
    
        longest_sequence = []
    
        if not subsequences:
            continue
        elif len(subsequences) == 1:
            longest_subsequence = subsequences[0]
        else:
            for ii in itertools.combinations(range(len(subsequences)), 2):
                subseq1 = subsequences[ii[0]]
                subseq2 = subsequences[ii[1]]
                prefix_i = 0
        
                while prefix_i < min(len(subseq1), len(subseq2)) and subseq1[prefix_i] == subseq2[prefix_i]:
                    prefix_i += 1
        
                if prefix_i == 0:
                    # No common prefix, cant clue these two subseqs
                    continue
        
                sequence = list(reversed(subseq1[prefix_i:])) + [subseq1[prefix_i - 1]] + subseq2[prefix_i:]
        
                if len(sequence) > len(longest_sequence):
                    longest_sequence = sequence
    
        if longest_sequence:
            sequences.append(longest_sequence)

    return sequences

In [ ]:
def simplify_sequences(sensor_instance_ind_dict, sequences, ma_window_size=4, precision=15):
    def _moving_average(a, n=3):
        ret = np.cumsum(a, dtype=float)
        ret[n:] = ret[n:] - ret[:-n]
        ret[:n-1] = np.array(a[:n-1]) * n
        return ret / n

    sequences_as_normal = []

    for s in sequences:
        sequence_as_normal = list(map(lambda si_ind: sensor_instance_ind_dict[si_ind].normal, s))
        sequences_as_normal.append(sequence_as_normal)

    sequences_simplified = []
    
    for sequence_as_normal, sequence in zip(sequences_as_normal, sequences):
        sequence_as_normal_diffs = np.array(sequence_as_normal) - np.hstack([[sequence_as_normal[0]], sequence_as_normal[:-1]])
        sequence_as_normal_diffs = (sequence_as_normal_diffs + 180) % 360 - 180
        sequence_as_normal_diffs_ma = _moving_average(sequence_as_normal_diffs, ma_window_size)
        
        shuttle_map = np.zeros_like(sequence_as_normal_diffs_ma, dtype='i')
        shuttles = dict()
        current_normal_diff = None
        insensitivity_lock = 0
        
        for i in range(ma_window_size, sequence_as_normal_diffs_ma.shape[0]):
            if current_normal_diff is None or abs(current_normal_diff - sequence_as_normal_diffs_ma[i]) > precision / 2:
                
                if insensitivity_lock == 0:
                    current_normal_diff = sequence_as_normal_diffs_ma[i]
    
                    if i > 0:
                        assert shuttle_map[i-1] == 0
                        shuttle_id = len(shuttles) + 1
                        shuttles[shuttle_id] = (i - 1, -1)
                        shuttle_map[i-1] = shuttle_id
                        sequence_as_normal_diffs_ma[i-1] = sequence_as_normal_diffs_ma[i]
    
                    if i < shuttle_map.shape[0] - 1:
                        assert shuttle_map[i+1] == 0
                        shuttle_id = len(shuttles) + 1
                        shuttles[shuttle_id] = (i + 1, +1)
                        shuttle_map[i+1] = shuttle_id
                        sequence_as_normal_diffs_ma[i+1] = sequence_as_normal_diffs_ma[i]
                    
                    insensitivity_lock += 3
        
            insensitivity_lock = max(0, insensitivity_lock - 1)

        while shuttles:
            dead_shuttles = []
            
            for shuttle_id, (shuttle_pos, shuttle_step) in shuttles.items():
                assert shuttle_map[shuttle_pos] == shuttle_id
                shuttle_map[shuttle_pos] = 0
                new_pos = shuttle_pos + shuttle_step
        
                if new_pos < 0 or new_pos >= shuttle_map.shape[0]:
                    dead_shuttles.append(shuttle_id)
                elif shuttle_map[new_pos] != 0:
                    rival_shuttle_id = shuttle_map[new_pos]
                    assert shuttles[rival_shuttle_id][1] == -shuttle_step
                    dead_shuttles.append(shuttle_id)
                    dead_shuttles.append(rival_shuttle_id)
                else:
                    shuttle_map[new_pos] = shuttle_id
                    shuttles[shuttle_id] = (new_pos, shuttle_step)
                    sequence_as_normal_diffs_ma[new_pos] = sequence_as_normal_diffs_ma[shuttle_pos - shuttle_step]
        
            for shuttle_id in dead_shuttles:
                del shuttles[shuttle_id]
        
        corner_indices = (1 + (np.argwhere(sequence_as_normal_diffs_ma[1:] != sequence_as_normal_diffs_ma[:-1]))).ravel()
        sequence_simplified = []
    
        for i in itertools.chain.from_iterable([[0], corner_indices, [sequence_as_normal_diffs_ma.shape[0] - 1]]):
            sequence_simplified.append(sequence[i])

        if sequence_simplified:
            sequences_simplified.append(sequence_simplified)

    return sequences_simplified